In [460]:
import numpy as np

In [479]:
class Investment:

    def __init__(self, invested, round):
        self.invested = invested
        self.round = round
        self.shares = np.round(self.invested / self.round.price,6)
        self.share_type = self.round.share_type

class Investor:

    def __init__(self, name):
        self.name = name
        self.investments = []

    def new_investment(self, invested, round):
        investment = Investment(invested, round)
        self.investments.append(investment)

    def total_shares(self):
        shares = 0
        for i in self.investments: 
            shares += i.shares 
        return shares

    def shares_ownership(self):
        stake = {}
        for i in self.investments:
            if i.share_type in stake.keys():
                stake[i.share_type] += i.shares
            else:
                stake[i.share_type] = i.shares
        return stake
    
    def total_invested(self):
        invested = 0
        for i in self.investments:
            invested += i.invested 
        return invested

class Covenants:
    
    def __init__(self, covenants={}):
        self.liquidity_preference = covenants.get('liquidity_preference', 0)
           
class Round:

    def __init__(self, name, invested, valuation, total_shares, new_shares, type, covenants):
        self.name = name
        self.invested = invested
        self.valuation = valuation
        self.new_shares = new_shares
        self.share_type = type if type != 'common' else 'z'
        self.price = round(valuation/total_shares,6)
        self.covenants = covenants

class BookInvestor:
    def __init__(self, name, stake):
        self.name = name
        self.stake = np.round(stake / 100,4)

class Deal:

    def __init__(self, investment, stake, share_type):
        self.investment = investment
        self.stake = np.round(stake / 100,4)
        self.share_type = share_type
        self.book = []

    def add_investor(self, investor):
        self.book.append(investor)

class Company:
    
    def __init__(self, name, equity, shares):
        self.name = name

        self.incorporation = Round('incorporation', equity, equity, shares, shares, 'common', Covenants())

        self.founder = Investor('founder')
        self.founder.new_investment(equity, self.incorporation)

        self.investors = [self.founder]
        self.rounds = [self.incorporation]

    def current_stake(self):
        cap = {}
        for i in self.investors:
            last_round = self.rounds[-1]
            total_shares = self.total_shares()
            investor_shares = i.total_shares()
            cap[i.name] = {
                'stake': np.round(investor_shares / total_shares * 100, 2)
            }
        return cap
    
    def shares_ownership(self):
        aux = {}
        for i, r in enumerate(self.rounds[::-1]):
            if r.share_type not in aux.keys(): 
                aux[r.share_type] = {'shares': r.new_shares, 'invested': r.invested, 'investors': {}, 'liquidity_preference_due': 0}
            else:
                aux[r.share_type]['shares'] += r.new_shares
                aux[r.share_type]['invested'] += r.invested

        for i in self.investors:
            for a in i.investments:
                if a.share_type in aux.keys():
                    if i.name not in aux[a.share_type]['investors'].keys():
                        aux[a.share_type]['investors'][i.name] = {
                            'invested': a.invested,
                            'shares': a.shares,
                            'company_stake': np.round(a.shares/ self.total_shares(),2),
                            'share_type_stake': np.round(a.shares/aux[a.share_type]['shares'],2),
                            'liquidity_preference_due': np.round(a.invested * a.round.covenants.liquidity_preference,2),
                        }
                    else:
                        aux[a.share_type]['investors'][i.name]['invested'] += a.invested
                        aux[a.share_type]['investors'][i.name]['shares'] += a.shares
                        aux[a.share_type]['investors'][i.name]['company_stake'] = np.round(aux[a.share_type]['investors'][i.name]['shares'] / self.total_shares(),2)
                        aux[a.share_type]['investors'][i.name]['share_type_stake'] = np.round(aux[a.share_type]['investors'][i.name]['shares']/aux[a.share_type]['shares'],2)
                        aux[a.share_type]['investors'][i.name]['liquidity_preference_due'] += np.round(a.invested * a.round.covenants.liquidity_preference,2)
                    aux[a.share_type]['liquidity_preference_due'] += np.round(a.invested * a.round.covenants.liquidity_preference,2)
        return sorted(aux.items())

    def get_investor(self, name):
        for i in self.investors:
            if name == i.name:
                return i
        return None

    def add_investor(self, name):
        new_investor = Investor(name)
        self.investors.append(new_investor)
        return new_investor

    def get__or_add_investor(self, name):
        investor = self.get_investor(name)
        if investor is None:
            investor = self.add_investor(name)
        return investor

    def total_shares(self):
        return sum(r.new_shares for r in self.rounds)

    def last_valuation(self):
        return self.rounds[-1].valuation

    def new_round(self, name, deal, covenants):

        
        post_money_valuation = deal.investment / deal.stake
        pre_money_valuation = post_money_valuation -  deal.investment
        
        old_shares = self.total_shares()
        new_shares =  old_shares * deal.investment / pre_money_valuation
        
        round = Round(name, deal.investment, post_money_valuation, old_shares+new_shares , new_shares, deal.share_type, covenants = covenants)
        
        self.rounds.append(round)

        for i in deal.book:
            new_investor = self.get__or_add_investor(i.name)
            new_investor.new_investment(deal.investment * i.stake , round)
            
        return None
    
    def exit(self, valuation):
        investors = self.current_stake()
        val_to_dist = valuation
        for i in self.shares_ownership():
            to_pay = np.minimum(i[-1]['liquidity_preference_due'], val_to_dist) if i[0] != 'z' else val_to_dist
            for j in i[-1]['investors'].items():
                stake_value = j[-1]['company_stake'] * valuation
                liquidity_pref_floor = np.minimum(j[-1]['liquidity_preference_due'], to_pay * j[-1]['share_type_stake']) 
                total_paid = np.minimum(np.maximum(liquidity_pref_floor, stake_value), val_to_dist)  if i[0] != 'z' else to_pay * j[-1]['share_type_stake']
                val_to_dist -= total_paid
                if 'invested' in investors[j[0]].keys():
                    investors[j[0]]['invested'] += j[-1]['invested']
                else:
                    investors[j[0]]['invested'] = j[-1]['invested']
                if 'total_paid' in investors[j[0]].keys():
                    investors[j[0]]['total_paid'] += total_paid 
                else:
                     investors[j[0]]['total_paid'] = total_paid
                if 'stake_value' in investors[j[0]].keys():
                    investors[j[0]]['stake_value'] += stake_value 
                else:
                     investors[j[0]]['stake_value'] = stake_value
                
        for i,k in enumerate(investors):
            investors[k]['liquidity_preference_value'] = investors[k]['total_paid'] - investors[k]['stake_value']
            investors[k]['MOIC'] = investors[k]['total_paid']/investors[k]['invested']
        return investors

            
        

In [496]:
capital = 1e4
price_per_share = 0.01

company = Company('consorciei', capital, capital/price_per_share)
company.incorporation.valuation

10000.0

In [497]:
cap = company.current_stake()
cap

{'founder': {'stake': 100.0}}

In [498]:
deal = Deal(10000, 10, 'common')
deal.add_investor(BookInvestor('ze', 100))
company.new_round('seed', deal, Covenants({}))

deal = Deal(500000, 20, 'a')
deal.add_investor(BookInvestor('tera', 80))
deal.add_investor(BookInvestor('ze', 20))
covenants = Covenants({'liquidity_preference': 1})
company.new_round('a', deal, covenants)

deal = Deal(1000000, 10, 'b')
deal.add_investor(BookInvestor('tera', 80))
deal.add_investor(BookInvestor('ze', 20))
covenants = Covenants({'liquidity_preference': 2})
company.new_round('b', deal, covenants)

In [499]:
company.current_stake()

{'founder': {'stake': 64.8}, 'ze': {'stake': 12.8}, 'tera': {'stake': 22.4}}

In [500]:
company.shares_ownership()

[('a',
  {'shares': 277777.77777777775,
   'invested': 500000,
   'investors': {'ze': {'invested': 100000.0,
     'shares': 55555.555556,
     'company_stake': 0.04,
     'share_type_stake': 0.2,
     'liquidity_preference_due': 100000.0},
    'tera': {'invested': 400000.0,
     'shares': 222222.222222,
     'company_stake': 0.14,
     'share_type_stake': 0.8,
     'liquidity_preference_due': 400000.0}},
   'liquidity_preference_due': 500000.0}),
 ('b',
  {'shares': 154320.98765432095,
   'invested': 1000000,
   'investors': {'ze': {'invested': 200000.0,
     'shares': 30864.197531,
     'company_stake': 0.02,
     'share_type_stake': 0.2,
     'liquidity_preference_due': 400000.0},
    'tera': {'invested': 800000.0,
     'shares': 123456.790123,
     'company_stake': 0.08,
     'share_type_stake': 0.8,
     'liquidity_preference_due': 1600000.0}},
   'liquidity_preference_due': 2000000.0}),
 ('z',
  {'shares': 1111111.111111111,
   'invested': 20000.0,
   'investors': {'founder': {'in

In [508]:
# sanity check
exit_multiple = 0.5
exit_valuation = company.last_valuation() * exit_multiple
val_to_dist = exit_valuation
paid_out = 0
print('exit valuation: ', exit_valuation)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
for i in company.shares_ownership():
    print('share class: ', i[0] if i[0] != 'z' else 'common')
    to_pay = np.minimum(i[-1]['liquidity_preference_due'], val_to_dist) if i[0] != 'z' else val_to_dist
    # print('to_pay: ', to_pay)
    for j in i[-1]['investors'].items():
        print('   investor: ', j[0])
        # print('      company_stake: ', j[-1]['company_stake'])
        stake_value = j[-1]['company_stake'] * exit_valuation
        # print('      stake_value: ', stake_value)
        liquidity_pref_floor = np.minimum(j[-1]['liquidity_preference_due'], to_pay * j[-1]['share_type_stake']) 
        # print('      liquidity_pref_floor: ', liquidity_pref_floor)
        total_paid = np.minimum(np.maximum(liquidity_pref_floor, stake_value), val_to_dist)  if i[0] != 'z' else to_pay * j[-1]['share_type_stake']
        print('      total_paid: ', total_paid)
        print('      paid_due_to_stake: ', stake_value)
        print('      paid_due_to_pref: ', total_paid-stake_value)
        paid_out += total_paid
        val_to_dist -= total_paid
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('total paid: ', paid_out)

exit valuation:  5000000.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
share class:  a
   investor:  ze
      total_paid:  200000.0
      paid_due_to_stake:  200000.0
      paid_due_to_pref:  0.0
   investor:  tera
      total_paid:  700000.0000000001
      paid_due_to_stake:  700000.0000000001
      paid_due_to_pref:  0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
share class:  b
   investor:  ze
      total_paid:  400000.0
      paid_due_to_stake:  100000.0
      paid_due_to_pref:  300000.0
   investor:  tera
      total_paid:  1600000.0
      paid_due_to_stake:  400000.0
      paid_due_to_pref:  1200000.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
share class:  common
   investor:  founder
      total_paid:  1890000.0
      paid_due_to_stake:  3250000.0
      paid_due_to_pref:  -1360000.0
   investor:  ze
      total_paid:  210000.0
      paid_due_to_stake:  350000.00000000006
      paid_due_to_pref:  -140000.00000000006
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [509]:
company.exit(company.last_valuation() * exit_multiple)

{'founder': {'stake': 64.8,
  'invested': 10000.0,
  'total_paid': 1890000.0,
  'stake_value': 3250000.0,
  'liquidity_preference_value': -1360000.0,
  'MOIC': 189.0},
 'ze': {'stake': 12.8,
  'invested': 310000.0,
  'total_paid': 810000.0,
  'stake_value': 650000.0,
  'liquidity_preference_value': 160000.0,
  'MOIC': 2.6129032258064515},
 'tera': {'stake': 22.4,
  'invested': 1200000.0,
  'total_paid': 2300000.0,
  'stake_value': 1100000.0,
  'liquidity_preference_value': 1200000.0,
  'MOIC': 1.9166666666666667}}